In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
import tensorflow as tf
from tensorflow.python.client import device_lib
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(device_lib.list_local_devices() )


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14314282817620823300
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14410383360
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4300859329806922105
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [4]:
## 변수
PATH = '/content/drive/MyDrive/Sign_Remaster/Sign_Language_Remaster'

In [5]:
def gan_load_and_preprocess_data(folder_path, target_frame_count=90, image_size=(128, 128), test_size=0.2, random_seed=42):
    data = []


    for filename in os.listdir(folder_path):
        if filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            cap = cv2.VideoCapture(video_path)
            frames = []

            while cap.isOpened() and len(frames) < target_frame_count:
                ret, frame = cap.read()
                if not ret:
                    break
                resized_frame = cv2.resize(frame, image_size)
                frames.append(resized_frame)

            # 프레임 수가 부족하면 패딩을 추가
            while len(frames) < target_frame_count:
                frames.append(np.zeros_like(frames[0]))

            frames = np.array(frames)

            data.append(frames)

            cap.release()
    data = np.array(data) / 255.0
    return data
data = gan_load_and_preprocess_data(f'{PATH}/data/hello')


In [6]:
data.shape

(50, 90, 128, 128, 3)

In [7]:

target_lable = 'hello'
def gan_load_and_preprocess_data(folder_path, target_frame_count=90, image_size=(128, 128), test_size=0.2, random_seed=42):
    data = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            cap = cv2.VideoCapture(video_path)
            frames = []

            while cap.isOpened() and len(frames) < target_frame_count:
                ret, frame = cap.read()
                if not ret:
                    break
                resized_frame = cv2.resize(frame, image_size)
                # RGB를 그레이스케일로 변환
                gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
                frames.append(gray_frame)

            # 프레임 수가 부족하면 패딩을 추가
            while len(frames) < target_frame_count:
                frames.append(np.zeros_like(frames[0]))

            frames = np.array(frames)

            data.append(frames)

            cap.release()
    data = np.array(data) / 255.0
    # 채널 차원 추가 (흑백으로 변환하면 채널이 1이 됨)
    data = np.expand_dims(data, axis=-1)
    return data
# 데이터 로드 및 전처리
data_dir = f'{PATH}/data/{target_lable}'
videos = gan_load_and_preprocess_data(data_dir)     # videos.shape ==(50, 90, 128, 128, 3)
print('데이터 전처리 완료')

# 생성자 정의
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim, activation='relu'))
    model.add(Dense(90*128*128*3, activation='sigmoid'))
    model.add(Reshape((90, 128, 128, 3)))
    return model

# 판별자 정의
def build_discriminator(input_shape):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(1, activation='sigmoid'))
    return model





데이터 전처리 완료


In [8]:
# GAN 모델 구성
latent_dim = 1
generator = build_generator(latent_dim)
discriminator = build_discriminator((90, 128, 128, 1))



ResourceExhaustedError: ignored

In [ ]:
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
discriminator.trainable = False

gan_input = Input(shape=(latent_dim,))
x = generator(gan_input)
gan_output = discriminator(x)

gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

# GAN 모델 학습
epochs = 10000
batch_size = 1

In [ ]:
print('모델 학습 시작')
for e in tqdm(range(epochs)):
    for _ in range(videos.shape[0] // batch_size):
        # 실제 동영상 데이터 배치 추출
        idx = np.random.randint(0, videos.shape[0], batch_size)
        real_videos = videos[idx]

        # 랜덤 노이즈 배치 생성
        noise = np.random.normal(0, 1, (batch_size, latent_dim))

        # 가짜 동영상 데이터 생성
        generated_videos = generator.predict(noise)

        # 실제 동영상과 가짜 동영상을 합치고 레이블 생성
        X = np.concatenate([real_videos, generated_videos])
        y_dis = np.zeros(2 * batch_size)
        y_dis[:batch_size] = 0.9  # 실제 동영상에 대한 레이블 부여

        # 판별자 학습
        discriminator.trainable = True
        d_loss = discriminator.train_on_batch(X, y_dis)

        # 생성자 학습
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        y_gen = np.ones(batch_size)
        discriminator.trainable = False
        g_loss = gan.train_on_batch(noise, y_gen)

    # 중간 과정 출력
    if e % 100 == 0:
        print(f"Epoch :{e}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}")

        # 생성된 동영상 저장
        if e % 1000 == 0:
            generated_videos = generated_videos * 255.0  # 정규화를 복원
            for i in range(generated_videos.shape[0]):
                np.save(f"{PATH}/created_data/epoch_{e}_{target_lable}_{i}.npy", generated_videos[i])



In [ ]:
# 학습된 모델 저장
generator.save(f"{PATH}/model/generator_model.h5")
discriminator.save(f"{PATH}/model/discriminator_model.h5")
gan.save(f"{PATH}/model/gan_model.h5")